In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from datetime import datetime
from pathlib import Path
import csv

In [2]:
# Parcing online stats from Steam game url using Selenium
def get_online_stats(url: str) -> tuple[str, int]:
    try:
        options = Options()
        options.add_argument("--headless")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(3)
        soup = bs(driver.page_source, "html.parser")
    except Exception as e:
        print(f"Произошла ошибка: {e}")
    finally:
        driver.quit()

    game_name = str(soup.find('div', class_='apphub_AppDetails').find('div', class_='apphub_AppName ellipsis').text)
    div_online_stats = soup.find('div', class_='apphub_Stats').find('span', class_='apphub_NumInApp')
    online_stats = int(div_online_stats.text.replace(',','').replace(' в игре', ''))
    print(f"Name: {game_name}, Online: {online_stats}")
    
    return game_name, online_stats

In [3]:
# Write online data to a new .csv
def write_online_stats_to_csv(game_name, online_stats):
    online_stats_data = {
        "name" : f"{game_name}",
        "player_count" : online_stats,
        "time" : datetime.now().strftime("%Y-%m-%d %H:%M")
    }
    file = Path('online_stats.csv')
    df = pd.DataFrame([online_stats_data])
    df.to_csv('online_stats.csv', index=False, mode='a', header=not file.exists(), quoting=csv.QUOTE_NONNUMERIC)

In [4]:
# Find game name by appid from Kaggle dataset (https://www.kaggle.com/datasets/artermiloff/steam-games-dataset)
def find_game_by_appid(name: str):
    df = pd.read_csv("datasets/games_march2025_cleaned.csv")
    match = df[df['name'] == name]
    if not match.empty:
        url = f"https://steamcommunity.com/app/{match.iloc[0]['appid']}"
    
    game_name, online_stats = get_online_stats(url)
    write_online_stats_to_csv(game_name, online_stats)

In [5]:
# Lets do it for Counter-Strike 2
find_game_by_appid("Counter-Strike 2")
df = pd.read_csv("online_stats.csv")
print(df)

Name: Counter-Strike 2, Online: 1130540
               name  player_count              time
0            Dota 2        360046  2025-05-05 11:58
1  Counter-Strike 2       1016157  2025-05-05 12:05
2  Counter-Strike 2       1168994  2025-05-06 21:02
3  Counter-Strike 2       1171001  2025-05-06 21:11
4  Counter-Strike 2       1172230  2025-05-06 21:16
5  Counter-Strike 2       1130540  2025-05-06 22:00
6  Counter-Strike 2       1130540  2025-05-06 22:03


In [ ]:
# Lets loop the code to run within 10 hours
start_time = time.time()
max_code_time = 10 * 60 * 60

hour = 0
while True:
    delta_time = time.time() - start_time
    if delta_time > max_code_time:
        print("10 hours are up. Finishing the code")
        break
    
    find_game_by_appid("Counter-Strike 2")
    
    hour += 1
    print(f"[{datetime.now()}] {hour} hour has passed")
    
    time.sleep(3600)

Name: Counter-Strike 2, Online: 1117033
[2025-05-06 22:04:23.620425] 1 hour has passed


In [ ]:
# Now lets plot the graph of online "Player Activity by Hour"